In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates
!pip install BeautifulSoup4
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 112kB 25.5MB/s eta 0:00:01
     |████████████████████████████████| 102kB 15.2MB/s ta 0:00:01
     |████████████████████████████████| 122kB 25.1MB/s eta 0:00:01
Libraries imported.


In [3]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(data, 'lxml')

table_contents=[]

table=soup.find('table')
table

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
# Dropping the rows where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Combining the neighborhoods with same Postalcode
df2 = df1.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2['Neighborhood'] = np.where(df2['Neighborhood'] == 'Not assigned',df2['Borough'], df2['Neighborhood'])

df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df2.rename(columns={'PostalCode':'Postalcode'},inplace=True)
lat_lon.rename(columns={'Postal Code':'Postalcode'},inplace=True)
lat_lon.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df3 = pd.merge(df2,lat_lon,on='Postalcode')
df3.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [8]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


In [14]:
address = ' Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of  Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

The geograpical coordinate of  Toronto are 43.65238435, -79.38356765.


 # I want to simplify the above map and segment and cluster only the neighborhoods in the centre of Toronto. 

In [12]:
centre_data = df4[df4['Borough'] == 'Central Toronto'].reset_index(drop=True)
centre_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


# Create map of Centre of Toronto using latitude and longitude values


In [15]:
address = '  Central Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Centre of  Toronto are {}, {}.'.format(latitude, longitude))

map_centre = folium.Map(location=[latitude, longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(centre_data['Latitude'],centre_data['Longitude'],centre_data['Borough'],centre_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_centre)
map_centre

The geograpical coordinate of Centre of  Toronto are 43.663461999999996, -79.39775965337452.


# USE FOURSQUARE API TO EXPLORE NEIGHBORHOODS AND SEGMENT

In [16]:
CLIENT_ID = 'YP03FATPA3WT3GI4K5D5EC2XOTUE3YKFYULHSR1DBJSIKK5M' # your Foursquare ID
CLIENT_SECRET = 'FQTXNXDDJYMZVYAUHCKY5TVVF1WZPJKLDMI411XXAGYOHV2P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [17]:
centre_data.loc[0, 'Neighborhood']

#Get the neighborhood's latitude and longitude values.

neighborhood_latitude = centre_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = centre_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = centre_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


# Now, let's get the top 100 venues that are in Lawrence Park  within a radius of 1000 meters.

In [21]:
LIMIT=100
radius=1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '609ead31c7b9444485560bab'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.73702050900001,
    'lng': -79.37635882105461},
   'sw': {'lat': 43.71902049099999, 'lng': -79.40122137894538}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': '

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Now we are ready to clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Granite Club,Gym / Fitness Center,43.733043,-79.381986
2,Tim Hortons,Coffee Shop,43.727324,-79.379563
3,Glendon Bookstore,Bookstore,43.727024,-79.378976
4,Glendon Forest,Trail,43.727226,-79.378413


In [25]:
print('{} venues were returned by Foursquare around Lawrence Park .'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare around Lawrence Park .


# Let's create a function to repeat the same process to all the neighborhoods in Centre of Toronto

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#create a new dataframe called manhattan_venues.¶

centre_venues=getNearbyVenues(names=centre_data['Neighborhood'],
                                   latitudes=centre_data['Latitude'],
                                   longitudes=centre_data['Longitude']
                                  )
#Let's check the size of the resulting dataframe

print(centre_venues.shape)
centre_venues.head()

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
(108, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


In [29]:
centre_venues.groupby('Neighborhood').count()

#Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories in the centre of Toronto.'.format(len(centre_venues['Venue Category'].unique())))

There are 63 uniques categories in the centre of Toronto.


## Analyze Each Neighborhood

In [32]:
# one hot encoding
centre_onehot = pd.get_dummies(centre_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
centre_onehot['Neighborhood'] = centre_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [centre_onehot.columns[-1]] + list(centre_onehot.columns[:-1])
centre_onehot = centre_onehot[fixed_columns]

centre_onehot.head()

centre_grouped = centre_onehot.groupby('Neighborhood').mean().reset_index()
centre_grouped
num_top_venues = 5
for hood in centre_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = centre_grouped[centre_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
              venue  freq
0    Sandwich Place  0.09
1      Dessert Shop  0.09
2  Sushi Restaurant  0.06
3       Coffee Shop  0.06
4       Pizza Place  0.06


----Davisville North----
                  venue  freq
0     Food & Drink Shop  0.14
1                 Hotel  0.14
2        Sandwich Place  0.14
3        Breakfast Spot  0.14
4  Gym / Fitness Center  0.14


----Forest Hill North & West----
                 venue  freq
0                Trail  0.25
1        Jewelry Store  0.25
2     Sushi Restaurant  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Lawrence Park----
                venue  freq
0            Bus Line  0.33
1                Park  0.33
2         Swim School  0.33
3  Light Rail Station  0.00
4         Pizza Place  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Restaurant  0.33
1               Lawyer  0.33
2                 Park  0.33
3  Rental Car Location  0.00
4  Indie Movie Theater  0.00


---

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = centre_grouped['Neighborhood']

for ind in np.arange(centre_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(centre_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Café,Coffee Shop,Thai Restaurant,Gym,Sushi Restaurant,Italian Restaurant,Indoor Play Area
1,Davisville North,Department Store,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Food & Drink Shop,Park,Hotel,Gourmet Shop,Gift Shop,Gas Station
2,Forest Hill North & West,Park,Jewelry Store,Trail,Sushi Restaurant,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
3,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Garden
4,"Moore Park, Summerhill East",Lawyer,Park,Restaurant,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Dessert Shop


## K-Mean Cluster

In [39]:
kclusters = 5

centre_grouped_clustering = centre_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(centre_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

centre_merged = centre_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
centre_merged = centre_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

centre_merged.head() # check the last columns!

#Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(centre_merged['Latitude'], centre_merged['Longitude'], centre_merged['Neighborhood'], centre_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
##cluster1
centre_merged.loc[centre_merged['Cluster Labels'] == 0, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,0,Department Store,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Food & Drink Shop,Park,Hotel,Gourmet Shop,Gift Shop,Gas Station
4,Central Toronto,0,Coffee Shop,Clothing Store,Salon / Barbershop,Mexican Restaurant,Park,Chinese Restaurant,Diner,Café,Rental Car Location,Restaurant
5,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Pharmacy,History Museum,Indian Restaurant,Donut Shop,Liquor Store,Middle Eastern Restaurant,Park
6,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Café,Coffee Shop,Thai Restaurant,Gym,Sushi Restaurant,Italian Restaurant,Indoor Play Area
8,Central Toronto,0,Coffee Shop,American Restaurant,Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pub,Pizza Place,Bank


In [42]:
##cluster2
centre_merged.loc[centre_merged['Cluster Labels'] == 1, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Home Service,Garden,Dessert Shop,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Fried Chicken Joint


In [44]:
##cluster3
centre_merged.loc[centre_merged['Cluster Labels'] == 2, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Garden


In [45]:
##cluster4
centre_merged.loc[centre_merged['Cluster Labels'] == 3, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio


In [46]:
##cluster5


centre_merged.loc[centre_merged['Cluster Labels'] == 4, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,4,Lawyer,Park,Restaurant,Yoga Studio,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Dessert Shop
